In [1]:
import torch

from envs.IBGym_mod_envs import IBGymQ
from dynamic_model.transforms import NormalizeTransform
from dynamic_programming.mdp_model import Policy

In [2]:
device = 'cpu'


model_path = 'dynamic_model/model'

model = torch.load(model_path).to(device)
model.eval()
model.look_ahead = 0

normalize_dataset = NormalizeTransform.load('dynamic_model/NormalizeInputConfigs.pkl')
normalize_dataset.to(device)

env = IBGymQ(q_model=model,device=device, setpoint=70, reward_type='classic', action_type='discrete', observation_type='include_past',
             reset_after_timesteps=1000, n_past_timesteps=model.seq_len, normalize_transformer=normalize_dataset)

policy_save_path = 'dynamic_programming/ib_policy.pkl'
policy = Policy.load(policy_save_path)

/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [ ]:
total_epochs = 0
episodes = 100
avg_ep_reward = 0

without_random_action_rewards = []
with_random_action_rewards = []
for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0

    done = False
    total_rewards = 0
    random_action = False
    while not done:
        if state in policy.state_to_index:
            action = policy.get_action(state)
        else:
            action = env.action_space.sample()
            random_action = True
        state, reward, done, info = env.step(action)


        epochs += 1

        total_rewards += reward
    if random_action:
        with_random_action_rewards.append(total_rewards/env.reset_after_timesteps)
        print('random action taken')
    else:
        without_random_action_rewards.append(total_rewards/env.reset_after_timesteps)
    print((total_rewards/env.reset_after_timesteps))
    avg_ep_reward += (total_rewards/env.reset_after_timesteps)
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average Reward per episode: {avg_ep_reward / episodes}")


random action taken
-530.2102529140392
-471.263854917924
random action taken
-569.7239221671186
random action taken
-2180.013610890101
random action taken
-506.5863166855896
-448.83579672322963
random action taken
-788.7215413489965
random action taken
-2319.300448328404
-1752.0902113710622
-2205.3190082704837
-2235.581046806837
random action taken
-438.21025809640224
random action taken
-533.0978396707367
random action taken
-480.66889145173093
random action taken
-393.52701117486095
random action taken
-632.5829849700092
-518.7529304456655
random action taken
-2326.023056263656
-1903.5394349533312
random action taken
-649.4952958313693
-881.1064024894706
-2467.162667197834
random action taken
-738.1779984222684
-462.74194103161904
random action taken
-508.3841996442079
-2154.1141719582733
-2495.605491433088
-452.43663557763915
-368.1401628561977
-2398.877144836338
random action taken
-544.5360040756833
-399.7041360859367
-2315.3695278628356
random action taken
-2562.1335122962373
ran

In [ ]:
print(f'Average Reward with random actions {sum(with_random_action_rewards)/len(with_random_action_rewards)}')
print(f'Average Reward without random actions {sum(without_random_action_rewards)/len(without_random_action_rewards)}')

In [ ]:
without_random_action_rewards